## Methodology:
  1.   Identify the appropriate dataset for this task

    1a. "2007 Baby Names" into search engine

    1b. Selected ssa.gov
  2.   Determine how to retrieve data from website

    2a. Looked for download link for dataset (Unavailable)

    2b. Decide to scrape website with BeautifulSoup
    
  3. Scrape data from website
  
    3a. Undestand HTMl Structure - Inspect element
    
    3b. Understand API Structure - Inspect element > Network > Header/Payload

    3c. Get response from API - fetch_baby_names() function

    3d. Parse API Request into Dataframe - fetch_baby_names() return value

  4. Pandas

    4a. Find set difference of 2007 names - 2020 names

    4b. Get top 10 names in set difference for each gender -
    [filter_rows_syntax].head(10)

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def fetch_baby_names(year):
    url = "https://www.ssa.gov/cgi-bin/popularnames.cgi"
    payload = {
        "year": year,
        "top": 100,
        "number": "p",
        "token" : "Submit"
    }
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    response = requests.post(url, data=payload, headers=headers)

    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table", {"summary":"Popularity for top 100"})

    #I think the "Submit" token doesn't always work and that the website limits how many requests you can do, hence the token field.
    if table is None:
        raise ValueError(f"Wait a bit before retrying.")

    rows = table.find_all("tr")[1:-1]  # Skip the header row. There's also a footer note that is also "tr" that needs to be removed.

    data = []
    for row in rows:
        cols = row.find_all("td")
        rank = int(cols[0].text.strip())
        boy_name = cols[1].text.strip()
        girl_name = cols[3].text.strip()
        data.append({"Rank": rank, "BoyName": boy_name, "GirlName": girl_name})
    return pd.DataFrame(data)

df_2007 = fetch_baby_names(2007)
df_2020 = fetch_baby_names(2020)

dropped_boy_names = set(df_2007['BoyName']) - set(df_2020['BoyName'])
dropped_girl_names = set(df_2007['GirlName']) - set(df_2020['GirlName'])

In [ ]:
#Top 10 declining boy names from 2007 to 2020, shown by name ranking in 2007
df_2007[df_2007['BoyName'].isin(dropped_boy_names)][['Rank', 'BoyName']].head(10).set_index('Rank')

,BoyName
Rank,
21,Tyler
31,Brandon
32,Gavin
39,Kevin
40,Evan
42,Zachary
45,Justin
54,Aidan
57,Diego


In [ ]:
#Top 10 declining girl names from 2007 to 2020,  shown by name ranking in 2007
df_2007[df_2007['GirlName'].isin(dropped_girl_names)][['Rank', 'GirlName']].head(10).set_index('Rank')

,GirlName
Rank,
13,Ashley
14,Alyssa
19,Alexis
22,Brianna
24,Taylor
26,Kayla
28,Lauren
32,Jasmine
34,Julia
